<a href="https://colab.research.google.com/github/Vishnu-3101/BitcoinResearch/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 460, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 460 (delta 131), reused 124 (delta 91), pack-reused 269
Receiving objects: 100% (460/460), 126.19 KiB | 4.67 MiB/s, done.
Resolving deltas: 100% (233/233), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 1.8 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pynvml/nvml.py", line 1798, in _LoadNvmlLibrary
    nvmlLib = CDLL("libnvidia-ml.so.1")
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: libnvidia-ml.so.1: cannot open shared object file: No such file or directory

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/rapidsai-csp-utils/colab/pip-install.py", line 18, in <module>
    pynvml

In [3]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print("Training on device:", device)

Training on device: cpu


In [5]:
import pandas as pd
import numpy as np
# import cudf
import spacy
# tweets_data = cudf.read_csv('/content/drive/MyDrive/Research Paper/Dataset/new_data3.csv')
# grouped_tweets = cudf.read_csv('/content/drive/MyDrive/Research Paper/Dataset/new_cleaned.csv')
grouped_tweets = pd.read_csv('/content/drive/MyDrive/Research Paper/Dataset/new_cleaned.csv')

In [33]:
grouped_tweets['concatenated_text'] = grouped_tweets['concatenated_text'].apply(lambda x: x.lower())

In [6]:
import torch
import random
from transformers import BertTokenizer, BertModel
random_seed = 42
random.seed(random_seed)

# Set a random seed for PyTorch (for GPU as well)
torch.manual_seed(random_seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(random_seed)

In [38]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

In [35]:
encoding = tokenizer.batch_encode_plus(
    # grouped_tweets['concatenated_text'].values_host,                    # List of input texts
    grouped_tweets['concatenated_text'],                    # List of input texts
    padding=True,              # Pad to the maximum sequence length
    truncation=True,           # Truncate to the maximum sequence length if necessary
    return_tensors='pt',      # Return PyTorch tensors
    add_special_tokens=True    # Add special tokens CLS and SEP
)

In [39]:
input_ids = encoding['input_ids'].to(device)
attention_mask = encoding['attention_mask'].to(device)

In [13]:
# from sklearn import preprocessing

# input_ids = preprocessing.normalize(input_ids.cpu())
# attention_mask = preprocessing.normalize(attention_mask.cpu())

In [ ]:
outputs = model(input_ids[0:10], attention_mask=attention_mask[0:10])
word_embeddings = outputs.last_hidden_state
word_embeddings = torch.nn.functional.normalize(word_embeddings.cpu()).to(device)

In [17]:
# btc_data = cudf.read_csv('/content/drive/MyDrive/Research Paper/Dataset/BTC-USD.csv')
btc_data = pd.read_csv('/content/drive/MyDrive/Research Paper/Dataset/BTC-USD.csv')

In [18]:
btc_data['Date'] = pd.to_datetime(btc_data['Date'],errors='coerce')
btc_data['date'] = btc_data['Date'].dt.day_of_year

In [19]:
btc_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,date
0,2021-02-07,39250.191406,39621.835938,37446.152344,38903.441406,38903.441406,65500641143,38
1,2021-02-08,38886.828125,46203.929688,38076.324219,46196.464844,46196.464844,101467222687,39
2,2021-02-09,46184.992188,48003.722656,45166.960938,46481.105469,46481.105469,91809846886,40
3,2021-02-10,46469.761719,47145.566406,43881.152344,44918.183594,44918.183594,87301089896,41
4,2021-02-11,44898.710938,48463.468750,44187.761719,47909.332031,47909.332031,81388911810,42


In [20]:
# combined_data = cudf.merge(btc_data, grouped_tweets, on='date', how='inner')
combined_data = pd.merge(btc_data, grouped_tweets, on='date', how='inner')

In [21]:
combined_data.head()

,Date,Open,High,Low,Close,Adj Close,Volume,date,Unnamed: 0,concatenated_text
0,2021-02-07,39250.191406,39621.835938,37446.152344,38903.441406,38903.441406,65500641143,38,2,pretty sure suppose cent 09 lol dogeto1dollar ...
1,2021-02-08,38886.828125,46203.929688,38076.324219,46196.464844,46196.464844,101467222687,39,3,Crypto Space Keeps history Bitcoin BTC btc bit...
2,2021-02-09,46184.992188,48003.722656,45166.960938,46481.105469,46481.105469,91809846886,40,4,Seller alert 10 BTC sell bid 47750 imb1mj8cs1...
3,2021-02-10,46469.761719,47145.566406,43881.152344,44918.183594,44918.183594,87301089896,41,5,Blue Ridge Bank share halt NYSE bitcoin ATM an...
4,2021-02-13,47491.203125,48047.746094,46392.281250,47105.515625,47105.515625,70250456155,44,6,LTC BTC Bitcoin 240 come hour add I twitch liv...


In [22]:
x = word_embeddings
y = combined_data['Close'][0:10]

In [23]:
x_tensor = torch.tensor(x, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1).to(device)

<ipython-input-23-984a45ec02da>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_tensor = torch.tensor(x, dtype=torch.float32).to(device)


In [24]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your CNN model
class SimpleCNN(nn.Module):
    def __init__(self, input_channels, output_size):
        super(SimpleCNN, self).__init__()

        self.conv1 = nn.Conv1d(in_channels=input_channels, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.relu = nn.ReLU()

        # Calculate the number of features after convolutional layers
        # This depends on the input size and the kernel sizes used
        self.fc_input_size = self.calculate_fc_input_size(input_channels, [self.conv1, self.conv2])

        self.fc1 = nn.Linear(6080, 64)
        self.fc2 = nn.Linear(64, output_size)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        # print(x.shape)
        # Flatten the output before passing it to fully connected layers
        x = x.view(-1, 6080)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

    def calculate_fc_input_size(self, input_channels, conv_layers):
        # Calculate the number of features after convolutional layers
        # This is needed to determine the input size for fully connected layers
        with torch.no_grad():
            x = torch.zeros(1, input_channels, 768)
            for layer in conv_layers:
                x = layer(x)
            return x.view(1, -1).size(1)

# Set the input channels and output size based on your problem
input_channels = 512
output_size = 1  # Assuming a regression task for predicting Bitcoin prices

# Create an instance of the SimpleCNN model
model = SimpleCNN(input_channels, output_size).to(device)

# Print the model architecture
print(model)

# Assuming you have a DataLoader and a loss function defined
# For example:
# dataloader = torch.utils.data.DataLoader(...)
criterion = nn.MSELoss()

# Set up the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 2


SimpleCNN(
  (conv1): Conv1d(512, 16, kernel_size=(3,), stride=(1,))
  (conv2): Conv1d(16, 32, kernel_size=(3,), stride=(1,))
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU()
  (fc1): Linear(in_features=6080, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=1, bias=True)
)


In [25]:
from tqdm import tqdm
for epoch in range(num_epochs):
      # print(x_tensor[i],y_tensor[i])
      optimizer.zero_grad()
      outputs = model(x_tensor)
      loss = criterion(outputs, y_tensor)
      loss.backward()
      optimizer.step()

In [26]:
model.eval()
with torch.no_grad():
    predictions = model(x_tensor)

In [30]:
from sklearn.metrics import mean_squared_error
mse_selected = mean_squared_error(y_tensor, predictions)
print(f'Mean Squared Error: {mse_selected}')

Mean Squared Error: 2344862976.0


In [31]:
btc_train_data = combined_data[['Open','High','Low','Close','Adj Close','Volume']].iloc[0:10]
y_btc = combined_data['Close'][0:10]

In [ ]:
!pip install pyswarms

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 1.8 MB/s eta 0:00:00


In [32]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

svm_model_selected = SVR()
svm_model_selected.fit(btc_train_data, y_btc)

# Make predictions on the test set using the model with selected features
y_pred_selected = svm_model_selected.predict(btc_train_data)

# Evaluate the performance of the SVM regression model with feature selection
mse_selected = mean_squared_error(y_btc, y_pred_selected)
print(f'Mean Squared Error with Feature Selection: {mse_selected}')


Mean Squared Error with Feature Selection: 21760677.023079447
